In [1]:
import torch
import os
from tqdm import tqdm

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch import nn, optim
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
from PIL import Image
from time import time
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('data/training/labels-tab-csv.csv')
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
image_arr = np.asarray(train.iloc[:, 0])
label_arr = np.asarray(train.iloc[:, 1])

train_img = []
for img_name in tqdm(image_arr):
    # defining the image path
    image_path = 'data/training/' + str(img_name)
    # reading the image
    img = imread(image_path, as_gray=True)
    
  #  t = transforms.Resize(60, 171)
    img = resize(img, (60, 171),
                       anti_aliasing=True)

    # normalizing the pixel values
 #   img /= 255.0
    img = np.true_divide(img, 255) #avoid division error
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = label_arr


train_x.shape


100%|███████████████████████████████████████████████████████████████████████████████| 859/859 [00:05<00:00, 163.34it/s]


(859, 60, 171)

In [3]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((773, 60, 171), (773,)), ((86, 60, 171), (86,)))

In [4]:
train_x = train_x.reshape(773, 1, 60, 171)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format

train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)


train_y = train_y.to(torch.long)
# shape of training data
train_x.shape, train_y.shape
print(train_x[0].dtype)
print(train_y[0].dtype)

torch.float32
torch.int64


In [5]:
val_x = val_x.reshape(86, 1, 60, 171)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

val_y = val_y.to(torch.long)
# shape of validation data
val_x.shape, val_y.shape
print(val_x.dtype)
print(val_y.dtype)

torch.float32
torch.int64


In [22]:
class TrainDatasetFromImages(Dataset):
    def __init__(self, img, vals):
        self.img = img
        self.vals = vals
    def __getitem__(self, index):
        img1 = self.img[index]
        label1 = self.vals[index]
        label1 = torch.tensor([label1], dtype=torch.long)
        return (img1, label1)
        
    def __len__(self):
        return self.data_len

In [23]:
train_data = TrainDatasetFromImages(train_x, train_y)
print(train_data[0])
input_size = 10260
hidden_sizes = [250]
output_size = 11 #might be an issue

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.LogSoftmax(dim=1))

(tensor([[[0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
         [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]]]), tensor([3]))


In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(train_data))
images = images.reshape(images.shape[0], -1)
logps = model(images) 
print(images.shape, labels.shape)
loss = criterion(logps, labels)


optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.91)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_data:
        images = images.reshape(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_x)))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [ ]:

valloader = TrainDatasetFromImages('data/training/labels-tab-csv.csv')

correct_count, all_count = 0, 0
for images,labels in train_data:
  for i in range(len(labels)):
    img = images[i].reshape(1, 2520)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    #print(pred_label)
    
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))


